# Initial setup

In [1]:
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()
role = "arn:aws:iam::941656036254:role/service-role/AmazonSageMaker-ExecutionRole-20210904T193230"

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/sm-modelparallel-distribution-options'
print('Bucket:\n{}'.format(bucket))

pandas failed to import. Analytics features will be impaired or broken.


Bucket:
sagemaker-us-east-1-941656036254


In [ ]:
sagemaker.__version__   

In [10]:
data_url = "s3://sagemaker-us-east-1-941656036254/hymenoptera_data"

In [8]:
bucket

'sagemaker-us-east-1-941656036254'

## Train Script



In [ ]:
! pygmentize 1_sources/train_resnet.py

## Debugger Config

In [7]:
from sagemaker.pytorch import PyTorch
from sagemaker.debugger import (
    Rule,
    DebuggerHookConfig,
    TensorBoardOutputConfig,
    CollectionConfig,
    rule_configs,
    ProfilerRule
)

rules = [
    Rule.sagemaker(rule_configs.vanishing_gradient()),
    Rule.sagemaker(rule_configs.overfit()),
    Rule.sagemaker(rule_configs.overtraining()),
    Rule.sagemaker(rule_configs.poor_weight_initialization()),
    ProfilerRule.sagemaker(rule_configs.ProfilerReport()) # TODO: need to review different profiler rules https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-built-in-rules.html#profiler-report
]

hook_config = DebuggerHookConfig(
    hook_parameters={"train.save_interval": "100", "eval.save_interval": "10"}
)

# Remote Mode

In [17]:

from sagemaker.pytorch import PyTorch

instance_type = 'ml.p2.xlarge'
instance_count = 1
job_name = "pytorch-smdebug-1"

estimator = PyTorch(
          entry_point="train_resnet_sm.py", # Pick your train script
          source_dir='2_sources',
          role=role,
          instance_type=instance_type,
          sagemaker_session=sagemaker_session,
          image_uri="763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.10.2-gpu-py38-cu113-ubuntu20.04-sagemaker",
          instance_count=instance_count,
          hyperparameters={
              "batch-size":16,
              "num-epochs":1,
              "input-size" : 224,
              "feature-extract":False,
              "tb-s3-url": f"s3://{bucket}/{job_name}"
          },
          disable_profiler=False,
          rules=rules,
          debugger_hook_config=hook_config,
          base_job_name=job_name,
      )

In [18]:
estimator.fit(inputs={"train":f"{data_url}/train", "val":f"{data_url}/val"})

2022-05-22 15:25:19 Starting - Starting the training job...
2022-05-22 15:25:54 Starting - Preparing the instances for trainingVanishingGradient: InProgress
Overfit: InProgress
Overtraining: InProgress
PoorWeightInitialization: InProgress
ProfilerReport: InProgress
............
2022-05-22 15:27:55 Downloading - Downloading input data...
2022-05-22 15:28:26 Training - Downloading the training image.............................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
/opt/conda/lib/python3.8/site-packages/paramiko/transport.py:236: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
2022-05-22 15:33:14,755 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-05-22 15:33:14,785 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-05-22 15:33:14,794 sagemaker_pytorch_container.training INFO     In